# 12.01 - Requests: Consumir APIs y Web

**Autor:** Miguel Angel Vazquez Varela  
**Nivel:** Intermedio  
**Tiempo estimado:** 25 min

---

## ¿Qué aprenderás?

- Realizar peticiones HTTP con `requests`
- Entender status codes y cabeceras
- Parsear respuestas JSON
- Enviar parámetros y cabeceras personalizadas
- Gestionar errores de red correctamente

---

## 1. ¿Qué es HTTP y por qué importa?

Cuando un navegador carga una página, realiza una **petición HTTP**. Con `requests` hacemos lo mismo desde Python: pedimos datos a un servidor y procesamos la respuesta.

```
Cliente (Python)  ──GET──▶  Servidor
                  ◀──200──  Respuesta (JSON / HTML)
```

In [1]:
# pip install requests
import requests

# Petición GET básica a una API pública de datos de bicicletas
url = "https://api.citybik.es/v2/networks/bicimad"
response = requests.get(url)

print(f"Status code : {response.status_code}")
print(f"Tipo de contenido: {response.headers['Content-Type']}")

Status code : 200
Tipo de contenido: application/json


### Status codes más comunes

| Código | Significado |
|---|---|
| 200 | OK - todo bien |
| 201 | Created - recurso creado |
| 400 | Bad Request - error en la petición |
| 401 | Unauthorized - necesita autenticación |
| 403 | Forbidden - sin permiso |
| 404 | Not Found - recurso no existe |
| 429 | Too Many Requests - rate limit |
| 500 | Server Error - fallo del servidor |

---

## 2. Parsear JSON

In [2]:
# Convertir la respuesta a un diccionario Python
data = response.json()

# Explorar la estructura
print(type(data))
print(data.keys())

<class 'dict'>
dict_keys(['network'])


In [3]:
import pandas as pd

# Extraer las estaciones y convertir a DataFrame
stations = data['network']['stations']
df = pd.DataFrame(stations)

print(f"Estaciones encontradas: {len(df)}")
df[['name', 'free_bikes', 'empty_slots', 'timestamp']].head()

Estaciones encontradas: 633


,name,free_bikes,empty_slots,timestamp
0,377 - Metro Abrantes,3,20,2026-02-20T17:12:30.979167+00:00Z
1,507 - Seis - Sexta,18,4,2026-02-20T17:12:30.978868+00:00Z
2,192 - Avda. de los Toreros - Fco. Silvela,16,11,2026-02-20T17:12:30.978334+00:00Z
3,333 - Illescas - Camarena,3,18,2026-02-20T17:12:30.979114+00:00Z
4,3 - Plaza Conde Suchil,12,6,2026-02-20T17:12:30.977227+00:00Z


---

## 3. Parámetros de consulta (query params)

In [4]:
# Los params se añaden automáticamente a la URL como ?key=value
url = "https://api.citybik.es/v2/networks"
params = {'fields': 'id,name,location', 'country': 'ES'}

response = requests.get(url, params=params)

# Ver la URL completa generada
print(f"URL real: {response.url}")

networks = response.json()['networks']
print(f"Redes en España: {len(networks)}")
for net in networks[:5]:
    print(f"  {net['name']} - {net['location']['city']}")

URL real: https://api.citybik.es/v2/networks?fields=id%2Cname%2Clocation&country=ES
Redes en España: 797
  Abu Dhabi Careem BIKE - Abu Dhabi
  Accès Vélo - Saguenay
  Aksu - 阿克苏市 (Aksu City)
  Alba - Alba
  AlbaBici - Albacete


---

## 4. Cabeceras y autenticación

In [5]:
# Muchas APIs requieren identificarte con un User-Agent o API key
headers = {
    'User-Agent': 'DataPortfolio/1.0 (educational project)',
    'Accept': 'application/json',
}

response = requests.get(url, headers=headers)
print(f"Status con headers: {response.status_code}")

# APIs con autenticación por token (ejemplo conceptual)
# headers = {'Authorization': 'Bearer TU_API_KEY'}
# response = requests.get(url, headers=headers)

Status con headers: 200


---

## 5. Gestión de errores

In [6]:
def fetch_station_data(network_id: str) -> dict | None:
    """
    Obtiene datos de una red de bicicletas.

    Parameters
    ----------
    network_id : str
        ID de la red en CityBikes API

    Returns
    -------
    dict | None
        Datos de la red o None si hay error
    """
    url = f"https://api.citybik.es/v2/networks/{network_id}"

    try:
        response = requests.get(url, timeout=10)
        response.raise_for_status()  # Lanza error si status >= 400
        return response.json()

    except requests.exceptions.ConnectionError:
        print("Error: sin conexión a internet")
    except requests.exceptions.Timeout:
        print("Error: la petición tardó demasiado")
    except requests.exceptions.HTTPError as e:
        print(f"Error HTTP {e.response.status_code}: {e}")

    return None


# Probar con ID válido
data = fetch_station_data('bicimad')
if data:
    print(f"Red: {data['network']['name']}")

# Probar con ID inválido
data_error = fetch_station_data('red-que-no-existe-xyz')

Red: BiciMAD
Error HTTP 404: 404 Client Error: Not Found for url: https://api.citybik.es/v2/networks/red-que-no-existe-xyz


---

## Resumen

| Concepto | Código |
|---|---|
| GET básico | `requests.get(url)` |
| Con parámetros | `requests.get(url, params={...})` |
| Con cabeceras | `requests.get(url, headers={...})` |
| Parsear JSON | `response.json()` |
| Verificar status | `response.raise_for_status()` |
| Timeout | `requests.get(url, timeout=10)` |

---

## Ejercicio

Usa la API de CityBikes para obtener las estaciones de otra ciudad española (ej: `valenbisi` para Valencia) y crea un DataFrame con las columnas: `name`, `free_bikes`, `empty_slots`. Filtra solo las estaciones con bicicletas disponibles.

In [7]:
# Tu solución aquí

---

**Anterior:** [11.05 - Polars Time Series](../11_advanced_patterns/11_05_polars_timeseries.ipynb)  
**Siguiente:** [12.02 - BeautifulSoup](./12_02_beautifulsoup.ipynb)